In [99]:
import numpy as np
def fix_to_float(in_hex):
    # no possible for negative number conversion
    # this function is not for general case
    # output after lru must be positive
    # expected length of input is 2 
    mp = {'0':'0000',
          '1':'0001',
          '2':'0010',
          '3':'0011',
          '4':'0100',
          '5':'0101',
          '6':'0110',
          '7':'0111',
          '8':'1000',
          '9':'1001',
          'A':'1010',
          'B':'1011',
          'C':'1100',
          'D':'1101',
          'E':'1110',
          'F':'1111'}
    
    binary_fmt = mp[in_hex[0]] + mp[in_hex[1]]
    ans = 0
    
    for i in range(7,-1,-1):
        #print(2**(i-4))
        ans += 2**(i-4) if binary_fmt[7-i] == '1' else 0
        #print(ans)
        
    return ans
    #print(binary_fmt)

def pass_dk1():
    dk1 = np.empty((0,128),float)
    row = np.array([])
    dk1f = open('dk1.txt','r')
    line = dk1f.readline()
    while line:
        for i in range(128): 
            row = np.append(row,float(line))
            line = dk1f.readline()
        dk1 = np.append(dk1,np.expand_dims(row, axis=0),axis=0)
        row = np.array([])
    dk1f.close()
    return dk1

def pass_dk2():
    dk2 = np.empty((0,10),float)
    row = np.array([])
    dk2f = open('dk2.txt','r')
    line = dk2f.readline()
    while line:
        for i in range(10): 
            row = np.append(row,float(line))
            line = dk2f.readline()
        dk2 = np.append(dk2,np.expand_dims(row, axis=0),axis=0)
        row = np.array([])
    dk2f.close()
    return dk2

def pass_db1():
    db1 = np.array([])
    with open('db1.txt') as f:
        for line in f:
            db1 = np.append(db1,float(line))
            if 'str' in line:
                  break
    return db1

def pass_db2():
    db2 = np.array([])
    with open('db2.txt') as f:
        for line in f:
            db2 = np.append(db2,float(line))
            if 'str' in line:
                  break
    return db2


def get_float_out1():
    # try to read data generate by hardware
    dk1 = np.zeros(676)
    index = 0
    dk1f = open('1.txt','r')
    line = dk1f.readline()
    while line:
        for i in range(169): 
            #print(line[0:2])
            #print(line[2:4])
            #print(line[4:6])
            #print(line[6:8])
            dk1[index] = fix_to_float(line[0:2])
            dk1[index+1] = fix_to_float(line[2:4])
            dk1[index+2] = fix_to_float(line[4:6])
            dk1[index+3] = fix_to_float(line[6:8])
            index += 4
            line = dk1f.readline()
    return dk1

def get_float_out2():
    # try to read data generate by hardware
    dk2 = np.zeros(676)
    index = 0
    dk2f = open('2.txt','r')
    line = dk2f.readline()
    while line:
        for i in range(169): 
            #print(line[0:2])
            #print(line[2:4])
            #print(line[4:6])
            #print(line[6:8])
            dk2[index] = fix_to_float(line[0:2])
            dk2[index+1] = fix_to_float(line[2:4])
            dk2[index+2] = fix_to_float(line[4:6])
            dk2[index+3] = fix_to_float(line[6:8])
            index += 4
            line = dk2f.readline()
    return dk2



def pass_x(path):
    x = np.empty((0,28),float)
    row = np.array([])
    xf = open(path,'r')
    line = xf.readline()
    while line:
        for i in range(28): 
            row = np.append(row,float(line))
            line = xf.readline()
        x = np.append(x,np.expand_dims(row, axis=0),axis=0)
        row = np.array([])
    xf.close()
    return x

def pass_y(path):
    y = np.array([])
    with open(path,'r') as f:
        for line in f:
            y = np.append(y,float(line))
            if 'str' in line:
                  break
    return y

In [94]:
dk1 = get_float_out1()
dk2 = get_float_out2()


r1 = np.array([])
r2 = np.array([])
l1_out1 = np.empty((0,26),float)
l1_out2 = np.empty((0,26),float)
d1_index = 0
d2_index = 0


# layer 1 convolution
for i in range(26):
    for j in range(26):
        r1 = np.append(r1,dk1[d1_index])
        d1_index += 1
        r2 = np.append(r2,dk2[d2_index])
        d2_index += 1
    l1_out1 = np.append(l1_out1,np.expand_dims(r1, axis=0),axis=0)
    l1_out2 = np.append(l1_out2,np.expand_dims(r2, axis=0),axis=0)
    r1 = np.array([])
    r2 = np.array([])
    
    
# layer 2 max pooling
maxp1 = -1
maxp2 = -1
l2_out1 = np.empty((0,13),float)
l2_out2 = np.empty((0,13),float)
for i in range(0,25,2):
    for j in range(0,25,2):
        for m in range(2):
            for n in range(2):
                maxp1 = max(maxp1,l1_out1[i+m,j+n]) 
                maxp2 = max(maxp2,l1_out2[i+m,j+n])
        r1 = np.append(r1,maxp1)
        r2 = np.append(r2,maxp2)
        maxp1 = -1
        maxp2 = -1
    l2_out1 = np.append(l2_out1,np.expand_dims(r1, axis=0),axis=0)
    l2_out2 = np.append(l2_out2,np.expand_dims(r2, axis=0),axis=0)  
    r1 = np.array([])
    r2 = np.array([])

# layer 3 flatten
l3_out = np.array([])

for i in range(13):
    for j in range(13):
        l3_out = np.append(l3_out,l2_out1[i,j])
        l3_out = np.append(l3_out,l2_out2[i,j])

dk1 = pass_dk1()
dk2 = pass_dk2()
db1 = pass_db1()
db2 = pass_db2()
# layer 4 dense

l4_out = np.dot(l3_out,dk1)
for i in range(128):
    l4_out[i] = max(l4_out[i]+db1[i],0)


# layer 5 dense with softmax

l5_out = np.dot(l4_out,dk2)
for i in range(10):
    l5_out[i] = l5_out[i]+db2[i]


ans = np.exp(l5_out)/sum(np.exp(l5_out))
for i in ans:
    print("%.5f"%(i))

    
    
v = 0
ma = -1
for i in range(10):
    if ans[i]>ma:
        ma = ans[i]
        v = i

if y[v]>0.5:
    return True
else: 
    return False

0.10740
0.00000
0.00001
0.00006
0.00000
0.00702
0.00000
0.88497
0.00001
0.00051


In [100]:
print(pass_y('test_data_y/test_data_y_0.txt'))
print(pass_x('test_data_x/test_data_x_0.txt'))

[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0